In [29]:
# Step 1: Load the data from CSV file

import pandas as pd
import os

file_path = os.path.join("data", "Fincial", "KBANK.BK.csv")

df = pd.read_csv(file_path)
df = df.iloc[2:]
print(df.head())

        Price  Close   High    Low   Open    Volume
2  2024-12-11  154.5  156.5  154.0  156.0   9896700
3  2024-12-12  156.5  157.5  155.0  155.0  14244300
4  2024-12-13  158.0  158.5  156.0  157.0  12712800
5  2024-12-16  157.0  157.5  156.5  157.0   6012300
6  2024-12-17  153.5  157.5  153.5  157.0   7103900


In [30]:
print(df.describe())

             Price  Close   High    Low   Open   Volume
count           41     41     41     41     41       41
unique          41     20     19     17     18       41
top     2024-12-11  154.5  161.5  160.0  155.0  9896700
freq             1      5      6      6      7        1


In [31]:
# Step 2: Preprocessing the data
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df = df[['Close']]  

df.dropna(inplace=True)
df['Close'] = df['Close'].astype(float)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

processed_data = pd.DataFrame(scaled_data, columns=['Close'], index=df.index)

print("Processed Data Sample:\n", processed_data.head())

Processed Data Sample:
       Close
2  0.181818
3  0.363636
4  0.500000
5  0.409091
6  0.090909


In [36]:
from sklearn.model_selection import train_test_split
import numpy as np

# Ensure enough data points exist before proceeding
if len(processed_data) < 60:
    raise ValueError("Not enough data points for LSTM training. At least 60 required.")

# Convert to NumPy array
scaled_data = processed_data.values  

# Split data into training (80%) and testing (20%)
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

# Initialize lists
x_train, y_train, x_test, y_test = [], [], [], []

# Generate sequences for training
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])  # 60 previous days
    y_train.append(train_data[i, 0])  # Predict the next day's close price

# Convert to NumPy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Generate sequences for testing
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    y_test.append(test_data[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)

# Ensure shapes are correct before reshaping
print(f"x_train shape before reshape: {x_train.shape}")
print(f"x_test shape before reshape: {x_test.shape}")

# Reshape for LSTM input (samples, time steps, features)
if x_train.shape[0] > 0 and x_train.shape[1] > 0:
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
else:
    raise ValueError("x_train is empty after preprocessing. Check dataset size.")

if x_test.shape[0] > 0 and x_test.shape[1] > 0:
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
else:
    raise ValueError("x_test is empty after preprocessing. Check dataset size.")

print(f"x_train shape after reshape: {x_train.shape}")
print(f"x_test shape after reshape: {x_test.shape}")


ValueError: Not enough data points for LSTM training. At least 60 required.